# Qualidade de Dados com PySpark
## MBA - Implementação Prática das Dimensões de DQ

Este notebook demonstra como implementar as 6 dimensões de qualidade de dados usando PySpark com dados de alunos universitários.

In [14]:
# Configuração inicial do Spark
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = SparkSession.builder \
    .appName("QualidadeDados") \
    .config("spark.sql.adaptive.enabled", "true") \
    .getOrCreate()

print(f"Spark Version: {spark.version}")

Spark Version: 3.3.0


## 1. Carregando Dataset de Alunos

In [15]:
# Carregando dataset de alunos
df = spark.read.option("header", "true").option("inferSchema", "true").csv("/home/tavares/data/alunos.csv")

print(f"Total de registros: {df.count()}")
print(f"Colunas: {df.columns}")
df.show(5, truncate=False)

Total de registros: 51
Colunas: ['id_aluno', 'nome_completo', 'email', 'telefone', 'cpf', 'data_nascimento', 'curso', 'periodo', 'nota_final', 'data_matricula', 'status_aluno']
+--------+---------------+---------------------------+-----------+-----------+-------------------+-------------+----------+----------+-------------------+------------+
|id_aluno|nome_completo  |email                      |telefone   |cpf        |data_nascimento    |curso        |periodo   |nota_final|data_matricula     |status_aluno|
+--------+---------------+---------------------------+-----------+-----------+-------------------+-------------+----------+----------+-------------------+------------+
|1       |João Silva     |joao@email.com             |11999887766|12345678901|1995-03-15 00:00:00|Engenharia   |Noturno   |8.5       |2020-01-15 00:00:00|Ativo       |
|2       |Maria Santos   |maria.santos@gmail.com     |21987654321|98765432100|1992-07-22 00:00:00|Administração|Matutino  |9.2       |2020-02-20 00:00:

## 2. Dimensão 1: COMPLETUDE
### Identificando campos obrigatórios não preenchidos

In [16]:
# Análise de completude
total_registros = df.count()
print("=== ANÁLISE DE COMPLETUDE ===")

for coluna in df.columns:
    nulos = df.filter(col(coluna).isNull()).count()
    vazios = df.filter(col(coluna) == "").count()
    preenchidos = total_registros - nulos - vazios
    percentual = (preenchidos / total_registros) * 100
    print(f"{coluna:20} | {preenchidos:2}/{total_registros} | {percentual:5.1f}%")

=== ANÁLISE DE COMPLETUDE ===
id_aluno             | 51/51 | 100.0%
nome_completo        | 51/51 | 100.0%
email                | 42/51 |  82.4%
telefone             | 51/51 | 100.0%
cpf                  | 51/51 | 100.0%
data_nascimento      | 51/51 | 100.0%
curso                | 51/51 | 100.0%
periodo              | 51/51 | 100.0%
nota_final           | 51/51 | 100.0%
data_matricula       | 51/51 | 100.0%
status_aluno         | 51/51 | 100.0%


## 3. Dimensão 2: VALIDADE
### Validando formatos de email, CPF e regras acadêmicas

In [17]:
# Validações de formato
print("=== ANÁLISE DE VALIDADE ===")

# Email válido
emails_validos = df.filter(
    col("email").isNotNull() & 
    col("email").rlike(r"^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$")
).count()
print(f"Emails válidos: {emails_validos}/{total_registros} ({emails_validos/total_registros*100:.1f}%)")

# CPF válido (11 dígitos)
cpfs_validos = df.filter(
    col("cpf").isNotNull() & 
    col("cpf").rlike(r"^\d{11}$")
).count()
print(f"CPFs válidos: {cpfs_validos}/{total_registros} ({cpfs_validos/total_registros*100:.1f}%)")

# Telefone válido (10-11 dígitos)
telefones_validos = df.filter(
    col("telefone").isNotNull() & 
    col("telefone").rlike(r"^\d{10,11}$")
).count()
print(f"Telefones válidos: {telefones_validos}/{total_registros} ({telefones_validos/total_registros*100:.1f}%)")

# Notas válidas (0-10)
notas_validas = df.filter(
    col("nota_final").isNotNull() & 
    (col("nota_final") >= 0) & 
    (col("nota_final") <= 10)
).count()
print(f"Notas válidas: {notas_validas}/{total_registros} ({notas_validas/total_registros*100:.1f}%)")

=== ANÁLISE DE VALIDADE ===
Emails válidos: 33/51 (64.7%)
CPFs válidos: 48/51 (94.1%)
Telefones válidos: 50/51 (98.0%)
Notas válidas: 50/51 (98.0%)


## 4. Dimensão 3: UNICIDADE
### Detectando alunos duplicados

In [18]:
# Análise de duplicatas
print("=== ANÁLISE DE UNICIDADE ===")

# Duplicatas por ID
ids_duplicados = df.groupBy("id_aluno").count().filter(col("count") > 1).count()
print(f"IDs duplicados: {ids_duplicados}")

# Duplicatas por CPF
cpfs_duplicados = df.filter(col("cpf").isNotNull()).groupBy("cpf").count().filter(col("count") > 1).count()
print(f"CPFs duplicados: {cpfs_duplicados}")

# Duplicatas por email
emails_duplicados = df.filter(col("email").isNotNull()).groupBy("email").count().filter(col("count") > 1).count()
print(f"Emails duplicados: {emails_duplicados}")

# Taxa de duplicação geral
registros_unicos = df.dropDuplicates(["id_aluno"]).count()
taxa_duplicacao = ((total_registros - registros_unicos) / total_registros) * 100
print(f"Taxa de duplicação: {taxa_duplicacao:.1f}%")
print(f"Registros únicos: {registros_unicos}/{total_registros}")

=== ANÁLISE DE UNICIDADE ===
IDs duplicados: 1
CPFs duplicados: 3
Emails duplicados: 1
Taxa de duplicação: 2.0%
Registros únicos: 50/51


## 5. Dimensão 4: CONSISTÊNCIA
### Verificando regras de negócio acadêmicas

In [19]:
# Análise de consistência
print("=== ANÁLISE DE CONSISTÊNCIA ===")

# Data matrícula não pode ser no futuro
matriculas_futuras = df.filter(
    to_date(col("data_matricula"), "yyyy-MM-dd") > current_date()
).count()
print(f"Matrículas no futuro: {matriculas_futuras}")

# Status vs Nota (alunos inativos devem ter nota 0)
status_nota_ok = df.filter(
    ((col("status_aluno") == "Ativo") & (col("nota_final") > 0)) |
    ((col("status_aluno") == "Inativo") & (col("nota_final") == 0)) |
    (col("status_aluno") == "Pendente")
).count()
print(f"Status/Nota consistentes: {status_nota_ok}/{total_registros} ({status_nota_ok/total_registros*100:.1f}%)")

# Idade mínima para matrícula (16 anos)
idades_validas = df.filter(
    months_between(to_date(col("data_matricula"), "yyyy-MM-dd"), to_date(col("data_nascimento"), "yyyy-MM-dd")) >= 192
).count()
print(f"Idades válidas na matrícula: {idades_validas}/{total_registros} ({idades_validas/total_registros*100:.1f}%)")

=== ANÁLISE DE CONSISTÊNCIA ===
Matrículas no futuro: 0
Status/Nota consistentes: 51/51 (100.0%)
Idades válidas na matrícula: 50/51 (98.0%)


## 6. Dimensão 5: ACURÁCIA
### Detectando anomalias nos dados acadêmicos

In [20]:
# Análise de acurácia
print("=== ANÁLISE DE ACURÁCIA ===")

# Nomes com espaços extras ou caracteres especiais
nomes_problematicos = df.filter(
    col("nome_completo").rlike(r"^\s+|\s+$|\s{2,}|\d+")
).count()
print(f"Nomes problemáticos: {nomes_problematicos}")

# Nascimentos no futuro
nascimentos_futuros = df.filter(
    to_date(col("data_nascimento"), "yyyy-MM-dd") > current_date()
).count()
print(f"Nascimentos no futuro: {nascimentos_futuros}")

# Notas impossíveis (negativas ou > 10)
notas_impossivel = df.filter(
    (col("nota_final") < 0) | (col("nota_final") > 10)
).count()
print(f"Notas impossíveis: {notas_impossivel}")

# Idades impossíveis (< 16 ou > 80 anos)
idades_impossivel = df.filter(
    (months_between(current_date(), to_date(col("data_nascimento"), "yyyy-MM-dd")) < 192) |
    (months_between(current_date(), to_date(col("data_nascimento"), "yyyy-MM-dd")) > 960)
).count()
print(f"Idades impossíveis: {idades_impossivel}")

=== ANÁLISE DE ACURÁCIA ===
Nomes problemáticos: 2
Nascimentos no futuro: 0
Notas impossíveis: 1
Idades impossíveis: 1


## 7. Dimensão 6: TEMPESTIVIDADE
### Analisando atualidade dos dados acadêmicos

In [21]:
# Análise de tempestividade
print("=== ANÁLISE DE TEMPESTIVIDADE ===")

# Matrículas recentes (último ano)
data_limite_recente = date_sub(current_date(), 365)
matriculas_recentes = df.filter(
    to_date(col("data_matricula"), "yyyy-MM-dd") >= data_limite_recente
).count()
print(f"Matrículas recentes (1 ano): {matriculas_recentes}/{total_registros} ({matriculas_recentes/total_registros*100:.1f}%)")

# Matrículas antigas (mais de 5 anos)
data_limite_antiga = date_sub(current_date(), 1825)
matriculas_antigas = df.filter(
    to_date(col("data_matricula"), "yyyy-MM-dd") < data_limite_antiga
).count()
print(f"Matrículas antigas (>5 anos): {matriculas_antigas}/{total_registros} ({matriculas_antigas/total_registros*100:.1f}%)")

# Dados atualizados (simulando última atualização)
dados_atualizados = df.filter(
    col("status_aluno").isin(["Ativo", "Pendente"])
).count()
print(f"Registros ativos/atualizados: {dados_atualizados}/{total_registros} ({dados_atualizados/total_registros*100:.1f}%)")

=== ANÁLISE DE TEMPESTIVIDADE ===
Matrículas recentes (1 ano): 0/51 (0.0%)
Matrículas antigas (>5 anos): 12/51 (23.5%)
Registros ativos/atualizados: 44/51 (86.3%)


## 8. DASHBOARD DE KQIs
### Métricas consolidadas de qualidade

In [22]:
# Calculando KQIs principais
print("\n" + "="*60)
print("           DASHBOARD DE QUALIDADE - DADOS ACADÊMICOS")
print("="*60)

# Completude (campos obrigatórios)
campos_obrigatorios = df.filter(
    col("nome_completo").isNotNull() & (col("nome_completo") != "") &
    col("email").isNotNull() & (col("email") != "") &
    col("cpf").isNotNull() & (col("cpf") != "") &
    col("curso").isNotNull() & (col("curso") != "")
).count()
completude = (campos_obrigatorios / total_registros) * 100

# Validade (formatos corretos)
registros_validos = df.filter(
    col("email").rlike(r"^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$") &
    col("cpf").rlike(r"^\d{11}$") &
    (col("nota_final") >= 0) & (col("nota_final") <= 10)
).count()
validade = (registros_validos / total_registros) * 100

# Unicidade
unicidade = (registros_unicos / total_registros) * 100

# Consistência (regras de negócio)
consistencia = (status_nota_ok / total_registros) * 100

# Acurácia (dados corretos)
registros_acurados = total_registros - nomes_problematicos - nascimentos_futuros - notas_impossivel
acuracia = (registros_acurados / total_registros) * 100

# Tempestividade (dados atuais)
tempestividade = (dados_atualizados / total_registros) * 100

print(f"📊 Completude:     {completude:5.1f}% (Meta: >95%)")
print(f"✅ Validade:       {validade:5.1f}% (Meta: >90%)")
print(f"🔄 Unicidade:      {unicidade:5.1f}% (Meta: >99%)")
print(f"🔗 Consistência:   {consistencia:5.1f}% (Meta: >95%)")
print(f"🎯 Acurácia:       {acuracia:5.1f}% (Meta: >95%)")
print(f"⏰ Tempestividade: {tempestividade:5.1f}% (Meta: >80%)")
print("="*60)

# Score geral
score_geral = (completude + validade + unicidade + consistencia + acuracia + tempestividade) / 6
if score_geral >= 95:
    status = "🟢 EXCELENTE"
elif score_geral >= 80:
    status = "🟡 BOM"
else:
    status = "🔴 CRÍTICO"

print(f"Score Geral: {score_geral:.1f}% - {status}")


           DASHBOARD DE QUALIDADE - DADOS ACADÊMICOS
📊 Completude:       0.0% (Meta: >95%)
✅ Validade:        60.8% (Meta: >90%)
🔄 Unicidade:       98.0% (Meta: >99%)
🔗 Consistência:   100.0% (Meta: >95%)
🎯 Acurácia:        94.1% (Meta: >95%)
⏰ Tempestividade:  86.3% (Meta: >80%)
Score Geral: 73.2% - 🔴 CRÍTICO


## 9. DATA CLEANSING
### Aplicando correções nos dados acadêmicos

In [23]:
# Aplicando limpeza nos dados
print("=== LIMPEZA DE DADOS ACADÊMICOS ===")

df_limpo = df \
    .filter(col("id_aluno").isNotNull()) \
    .dropDuplicates(["id_aluno"]) \
    .withColumn("nome_completo", trim(col("nome_completo"))) \
    .filter(col("nome_completo").isNotNull() & (col("nome_completo") != "")) \
    .filter(col("email").rlike(r"^[\w\.-]+@[\w\.-]+\.[a-zA-Z]{2,}$")) \
    .filter((col("nota_final") >= 0) & (col("nota_final") <= 10)) \
    .filter(to_date(col("data_nascimento"), "yyyy-MM-dd") <= current_date()) \
    .filter(to_date(col("data_matricula"), "yyyy-MM-dd") <= current_date())

registros_limpos = df_limpo.count()
registros_removidos = total_registros - registros_limpos

print(f"Registros originais: {total_registros}")
print(f"Registros limpos: {registros_limpos}")
print(f"Registros removidos: {registros_removidos} ({registros_removidos/total_registros*100:.1f}%)")

print("\nDistribuição por curso (dados limpos):")
df_limpo.groupBy("curso").count().orderBy(desc("count")).show(10)

print("\nDistribuição por status (dados limpos):")
df_limpo.groupBy("status_aluno").count().show()

=== LIMPEZA DE DADOS ACADÊMICOS ===
Registros originais: 51
Registros limpos: 32
Registros removidos: 19 (37.3%)

Distribuição por curso (dados limpos):
+-----------+-----+
|      curso|count|
+-----------+-----+
| Engenharia|    3|
|   Biologia|    1|
| Literatura|    1|
|Arquitetura|    1|
| Matemática|    1|
|Odontologia|    1|
|     Física|    1|
|      Dança|    1|
|  Filosofia|    1|
| Computação|    1|
+-----------+-----+
only showing top 10 rows


Distribuição por status (dados limpos):
+------------+-----+
|status_aluno|count|
+------------+-----+
|    Pendente|    2|
|     Inativo|    1|
|       Ativo|   29|
+------------+-----+



## 10. Relatório Executivo
### Insights e recomendações

In [24]:
# Relatório executivo
print("\n" + "="*70)
print("              RELATÓRIO EXECUTIVO - QUALIDADE DE DADOS")
print("="*70)
print(f"Dataset: Sistema Acadêmico - Cadastro de Alunos")
print(f"Período de análise: {df.select(min('data_matricula')).collect()[0][0]} a {df.select(max('data_matricula')).collect()[0][0]}")
print(f"Total de registros analisados: {total_registros}")
print(f"Registros aprovados na limpeza: {registros_limpos} ({registros_limpos/total_registros*100:.1f}%)")

print("\n📊 PRINCIPAIS ACHADOS:")
print(f"• Emails inválidos: {total_registros - emails_validos} registros")
print(f"• CPFs inválidos: {total_registros - cpfs_validos} registros")
print(f"• Registros duplicados: {total_registros - registros_unicos} registros")
print(f"• Campos obrigatórios vazios: {total_registros - campos_obrigatorios} registros")
print(f"• Notas fora do padrão: {notas_impossivel} registros")

print("\n🎯 RECOMENDAÇÕES PRIORITÁRIAS:")
print("1. Implementar validação de email em tempo real no sistema")
print("2. Adicionar validação de CPF com dígito verificador")
print("3. Criar processo automático de detecção de duplicatas")
print("4. Estabelecer regras de consistência status vs nota")
print("5. Implementar auditoria de dados em tempo real")

print("\n💡 IMPACTO ESTIMADO:")
print(f"• Melhoria na confiabilidade: +{(95 - score_geral):.1f} pontos percentuais")
print(f"• Redução de retrabalho: ~{registros_removidos * 0.5:.0f} horas/mês")
print(f"• Melhoria na experiência do usuário: Significativa")

# Salvando dados limpos
df_limpo.coalesce(1).write.mode("overwrite").option("header", "true").csv("/tmp/alunos_limpos")
print("\n✅ Dados limpos salvos em: /tmp/alunos_limpos")

# Finalizando
spark.stop()
print("\n🏁 Análise concluída com sucesso!")


              RELATÓRIO EXECUTIVO - QUALIDADE DE DADOS
Dataset: Sistema Acadêmico - Cadastro de Alunos
Período de análise: 2019-01-15 00:00:00 a 2023-12-15 00:00:00
Total de registros analisados: 51
Registros aprovados na limpeza: 32 (62.7%)

📊 PRINCIPAIS ACHADOS:
• Emails inválidos: 18 registros
• CPFs inválidos: 3 registros
• Registros duplicados: 1 registros
• Campos obrigatórios vazios: 51 registros
• Notas fora do padrão: 1 registros

🎯 RECOMENDAÇÕES PRIORITÁRIAS:
1. Implementar validação de email em tempo real no sistema
2. Adicionar validação de CPF com dígito verificador
3. Criar processo automático de detecção de duplicatas
4. Estabelecer regras de consistência status vs nota
5. Implementar auditoria de dados em tempo real

💡 IMPACTO ESTIMADO:
• Melhoria na confiabilidade: +21.8 pontos percentuais
• Redução de retrabalho: ~10 horas/mês
• Melhoria na experiência do usuário: Significativa

✅ Dados limpos salvos em: /tmp/alunos_limpos

🏁 Análise concluída com sucesso!
